In [0]:
from pyspark.sql import SparkSession
!pip install yfinance
import yfinance as yf
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Project').getOrCreate()
from pyspark.sql.functions import when
from pyspark.sql.functions import max
from pyspark.sql.functions import greatest,col,lit

In [0]:
pd_df=spark.read.csv('/FileStore/shared_uploads/saisuraja@usf.edu/info_csv_2/part-00000-tid-5587999557315711952-211515bf-b4c6-4cef-9b6f-28e8f32371fe-317-1-c000.csv',inferSchema=True,header=True)

In [0]:
pd_df.select('longBusinessSummary').show(2)

+--------------------+
 longBusinessSummary|
+--------------------+
3M Company develo...|
A. O. Smith Corpo...|
+--------------------+
only showing top 2 rows

In [0]:
pd_df_new=pd_df.select('symbol','longBusinessSummary').dropna()

In [0]:
pd_df_new.count()
pd_df_new.select('longBusinessSummary').show(2)

+--------------------+
 longBusinessSummary|
+--------------------+
3M Company develo...|
A. O. Smith Corpo...|
+--------------------+
only showing top 2 rows

In [0]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,StopWordsRemover,RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol="longBusinessSummary", outputCol="words", pattern="\\W")
wordsData = regexTokenizer.transform(pd_df_new)

remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_data=remover.transform(wordsData)


In [0]:
from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=300, minCount=0, inputCol="filtered", outputCol="result")
model = word2Vec.fit(filtered_data)
result = model.transform(filtered_data)

In [0]:
result.show(2)

+------+--------------------+--------------------+--------------------+--------------------+
symbol| longBusinessSummary| words| filtered| result|
+------+--------------------+--------------------+--------------------+--------------------+
 MMM|3M Company develo...|, company, dev...|, company, dev...|[-0.0169542803692...|
 AOS|A. O. Smith Corpo...|[a, o, smith, cor...|[o, smith, corpor...|[-0.0014848106550...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
data_final = result.select("*").withColumn("id", monotonically_increasing_id())

In [0]:
data_final.filter("symbol=='AAPL'").show()

+------+--------------------+--------------------+--------------------+--------------------+---+
symbol| longBusinessSummary| words| filtered| result| id|
+------+--------------------+--------------------+--------------------+--------------------+---+
 AAPL|Apple Inc. design...|[apple, inc, desi...|[apple, inc, desi...|[-0.0123012896800...| 46|
+------+--------------------+--------------------+--------------------+--------------------+---+

In [0]:
def sim_cos(v1,v2):
    try:
        p = 2
        return float(v1.dot(v2))/float(v1.norm(p)*v2.norm(p))
    except:
        return 0

In [0]:
Cosine_List=[]
v1=result.select('result').collect()[46][0]
for i in range(0,result.count()):
  v2=result.select('result').collect()[i][0]
  Cosine_List.append((result.select('symbol').collect()[46][0],result.select('symbol').collect()[i][0],sim_cos(v1,v2)))

In [0]:
from pyspark.sql import Row
similiar_columns = ["ticker_1", "ticker_2", "cosine_similiarity"]
df_cosine_similiarity = spark.createDataFrame(data=Cosine_List, schema = similiar_columns)

In [0]:
df_cosine_similiarity.sort('cosine_similiarity',ascending =False).show()

+--------+--------+------------------+
ticker_1|ticker_2|cosine_similiarity|
+--------+--------+------------------+
 AAPL| AAPL|1.0000000000000002|
 AAPL| ABC|0.9942151298381819|
 AAPL| MMM|0.9898410198504841|
 AAPL| ZBRA| 0.989374439628936|
 AAPL| MTD|0.9865003086720777|
 AAPL| WST|0.9849962142868562|
 AAPL| DHR|0.9835846834446371|
 AAPL| WAT|0.9820954317361881|
 AAPL| PKI|0.9818181109803371|
 AAPL| CAH|0.9816821658222683|
 AAPL| ADI|0.9810000527946664|
 AAPL| AME|0.9807466791948543|
 AAPL| CHTR|0.9806825288023931|
 AAPL| CVS|0.9806215628322239|
 AAPL| ACN|0.9782307744274955|
 AAPL| PGR|0.9779812207002035|
 AAPL| QCOM|0.9766127630654166|
 AAPL| A|0.9755140760705141|
 AAPL| XYL|0.9744961777270764|
 AAPL| APH|0.9743711239514222|
+--------+--------+------------------+
only showing top 20 rows